In [0]:
tiers = ["bronze","silver","gold"]
adls_paths = {}

for x in tiers:
    adls_paths[x] = f"abfss://{x}@abychen.dfs.core.windows.net/"
    # container paths are stored in the dictionary
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]


In [0]:
import requests
# for API requests
import json
# for converting json to python objects
from datetime import date, timedelta

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"
# assigned start and end dates to url for fetching yesterdays data

try:
    response = requests.get(url)
    # getting earthquake data for yesterday using API
    response.raise_for_status()
    # checking if response is successful

    data = response.json().get("features",[])
    # getting the features from the response in GeoJSON format

    if not data:
        print("No data received")
    else:
        file_path = f"{bronze_adls}{start_date}_earthquake_data.json"
        json_data = json.dumps(data, indent=4)
        # converting geojson data to json

        dbutils.fs.put(file_path, json_data, overwrite=True)
        #writing to adls
        print(f"Data saved to {file_path}")

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")



In [0]:
output_data = {
    "start_date":start_date.isoformat(), # isoformat() converts date to string for sending the output data
    "end_date": end_date.isoformat(),
    "bronze_adls":file_path,
    "silver_adls":silver_adls,
    "gold_adls":gold_adls
}

dbutils.jobs.taskValues.set(key="bronze_output", value=output_data)